In [1]:
from google.colab import drive
import pandas as pd
# Montar Google Drive
drive.mount('/content/drive')

# Leer el archivo CSV
df = pd.read_csv('/content/drive/My Drive/Beca bootcamp/Food_and_Nutrition__.csv')

# Inspección inicial
print(df.head())
print(df.describe())
print(df.info())
print(df.isnull().sum())

Mounted at /content/drive
   Ages  Gender  Height  Weight     Activity Level Dietary Preference  \
0    25    Male     180      80  Moderately Active           Omnivore   
1    32  Female     165      65     Lightly Active         Vegetarian   
2    48    Male     175      95          Sedentary              Vegan   
3    55  Female     160      70        Very Active           Omnivore   
4    62    Male     170      85          Sedentary         Vegetarian   

   Daily Calorie Target  Protein  Sugar  Sodium  Calories  Carbohydrates  \
0                  2000      120  125.0    24.0      2020            250   
1                  1600       80  100.0    16.0      1480            200   
2                  2200      100  150.0    20.0      2185            300   
3                  2500      140  175.0    28.0      2680            350   
4                  2000       80  125.0    16.0      1815            250   

   Fiber  Fat                               Breakfast Suggestion  \
0   30.0  

# Parte 1: Preprocesamiento de Datos

**Limpieza de Datos:**
Tratar los valores nulos utilizando técnicas adecuadas (imputación, eliminación, etc.).

Manejar los outliers mediante técnicas de filtrado o transformación.

**Transformación de Columnas:**

Utilizar ColumnTransformer para aplicar transformaciones específicas a diferentes columnas.

Realizar codificación de variables categóricas utilizando técnicas como One-Hot Encoding.

Escalar las variables numéricas usando StandardScaler u otros métodos de normalización.

**Creación de Pipelines:**

Crear pipelines utilizando Pipeline de sklearn para automatizar el preprocesamiento de datos y asegurar la reproducibilidad.

Incluir todos los pasos de preprocesamiento en el pipeline.

# Parte 2: Selección de Técnica de Machine Learning

**Entrenamiento Inicial:**

Entrenar múltiples modelos de machine learning (por ejemplo, Regresión Lineal, KNN, Árbol de Decisión, Random Forest, XGBoost, LGBM).

Evaluar los modelos utilizando validación cruzada y seleccionar el modelo con el mejor rendimiento inicial.

**Comparación de Modelos:**


In [4]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings("ignore")

# ---- Limpieza de Datos ----
# Tratar valores nulos
num_cols = df.select_dtypes(include=['float64', 'int64']).columns
cat_cols = df.select_dtypes(include=['object']).columns

imputer_num = SimpleImputer(strategy='mean')
imputer_cat = SimpleImputer(strategy='most_frequent')

df[num_cols] = imputer_num.fit_transform(df[num_cols])
df[cat_cols] = imputer_cat.fit_transform(df[cat_cols])

# Manejar outliers mediante winsorización (ajustar valores extremos)
def winsorize(series, low=0.01, high=0.99):
    return series.clip(lower=series.quantile(low), upper=series.quantile(high))

for col in num_cols:
    df[col] = winsorize(df[col])

# ---- Transformación de Columnas ----
# Definir columnas categóricas y numéricas
categorical_cols = df.select_dtypes(include=['object']).columns
numerical_cols = df.select_dtypes(include=['float64', 'int64']).columns.drop("Calories")

# Preprocesadores
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Crear ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ]
)

# ---- División de Datos ----
X = df.drop(columns=['Calories'])
y = df['Calories']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ---- Creación de Pipelines y Modelos ----
models = {
    'Regresión Lineal': LinearRegression(),
    'KNN': KNeighborsRegressor(),
    'Árbol de Decisión': DecisionTreeRegressor(),
    'Random Forest': RandomForestRegressor(),
    'XGBoost': XGBRegressor(),
    'LightGBM': LGBMRegressor()
}

# Evaluación inicial
resultados = []
for name, model in models.items():
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', model)
    ])
    scores = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
    rmse_scores = np.sqrt(-scores)
    resultados.append({
        'Modelo': name,
        'RMSE Promedio': rmse_scores.mean(),
        'RMSE Std Dev': rmse_scores.std()
    })

# Crear un DataFrame con los resultados
resultados_df = pd.DataFrame(resultados).sort_values(by='RMSE Promedio')
print(resultados_df)

# ---- Selección del Mejor Modelo ----
mejor_modelo_nombre = resultados_df.iloc[0]['Modelo']
mejor_modelo = models[mejor_modelo_nombre]

pipeline_mejor = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', mejor_modelo)
])

# Entrenar el mejor modelo
pipeline_mejor.fit(X_train, y_train)

# Evaluar en el conjunto de prueba
y_pred = pipeline_mejor.predict(X_test)
rmse_test = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"Mejor modelo: {mejor_modelo_nombre}")
print(f"RMSE en el conjunto de prueba: {rmse_test:.4f}")



[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000178 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1463
[LightGBM] [Info] Number of data points in the train set: 1086, number of used features: 68
[LightGBM] [Info] Start training from score 2205.199541
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000162 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1468
[LightGBM] [Info] Number of data points in the train set: 1086, number of used features: 69
[LightGBM] [Info] Start training from score 2208.811999
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000169 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not e

Alparecer el modelo de Regresión Lineal fue el mejor en este caso, con un RMSE promedio más bajo durante la validación cruzada y un desempeño aceptable en el conjunto de prueba con un RMSE de 14.4298. Esto indica que la Regresión Lineal es adecuada para el problema planteado de predecir las calorías consumidas.

Observaciones:
LightGBM y XGBoost ofrecen resultados razonablemente buenos pero no superan el desempeño de la Regresión Lineal en este caso. Esto podría deberse a que la relación entre las variables es lineal, haciendo que los modelos complejos no agreguen valor significativo.

KNN, Árbol de Decisión y Random Forest tienen peores resultados, indicando que no capturan bien la estructura de los datos para este problema.

# Parte 3: Optimización de Hiperparámetros

**GridSearchCV:**

Implementar GridSearchCV para realizar una búsqueda exhaustiva de los mejores hiperparámetros para el modelo seleccionado.
Definir el espacio de búsqueda para los hiperparámetros relevantes.

**RandomizedSearchCV:**

Implementar RandomizedSearchCV para realizar una búsqueda aleatoria de los mejores hiperparámetros, especialmente útil si el espacio de búsqueda es grande.

**Optuna:**

Implementar Optuna para una optimización avanzada de los hiperparámetros, aprovechando técnicas como la optimización bayesiana y el pruning.

**Evaluación de Modelos Optimizados:**

Entrenar el modelo con los mejores hiperparámetros encontrados y evaluar su rendimiento en el conjunto de prueba.
Comparar el rendimiento del modelo optimizado con el modelo inicial.

In [7]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.impute import SimpleImputer
import numpy as np

# Definir las columnas numéricas y categóricas
numerical_features = X_train.select_dtypes(include=['float64', 'int64']).columns
categorical_features = X_train.select_dtypes(include=['object']).columns

# Crear un transformador para las columnas numéricas y categóricas
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),  # Normalizar las variables numéricas
        ('cat', Pipeline([
            ('imputer', SimpleImputer(strategy='most_frequent')),  # Imputar valores faltantes en categorías
            ('onehot', OneHotEncoder(handle_unknown='ignore'))    # Aplicar One-Hot Encoding
        ]), categorical_features)
    ])

# Crear el pipeline con el preprocesador y el modelo de regresión lineal
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regression', LinearRegression())
])

# Definir el espacio de búsqueda de hiperparámetros
param_grid = {
    'regression__fit_intercept': [True, False],
    'regression__copy_X': [True, False]
}

# Crear el GridSearchCV
grid_search = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

# Ajustar el modelo
grid_search.fit(X_train, y_train)

# Imprimir los mejores hiperparámetros y el rendimiento
print(f"Mejores hiperparámetros: {grid_search.best_params_}")
print(f"Mejor puntuación de validación (neg_mse): {grid_search.best_score_}")

# Evaluar el modelo optimizado
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"RMSE del modelo optimizado: {rmse}")




Mejores hiperparámetros: {'regression__copy_X': True, 'regression__fit_intercept': True}
Mejor puntuación de validación (neg_mse): -151.0864223294411
RMSE del modelo optimizado: 14.429764710724577


In [25]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np

# Supongamos que tienes un DataFrame X_train y X_test con datos numéricos y categóricos, y y_train, y_test como etiquetas
# Definir las columnas numéricas y categóricas
numerical_cols = X_train.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = X_train.select_dtypes(include=['object']).columns

# Crear el transformador para las columnas
preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline([
            ('imputer', SimpleImputer(strategy='mean')),
            ('scaler', StandardScaler())
        ]), numerical_cols),
        ('cat', Pipeline([
            ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
            ('onehot', OneHotEncoder(handle_unknown='ignore'))
        ]), categorical_cols)
    ])

# Crear el pipeline completo con el preprocesamiento y la regresión lineal
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regression', LinearRegression())
])

# Definir el espacio de búsqueda para los hiperparámetros
param_grid = {
    'regression__fit_intercept': [True, False],
    'regression__copy_X': [True, False]
}

# Definir RandomizedSearchCV
random_search = RandomizedSearchCV(pipeline, param_distributions=param_grid, n_iter=10, cv=5, verbose=2, n_jobs=-1)

# Ajustar el modelo
random_search.fit(X_train, y_train)

# Imprimir los mejores hiperparámetros y el rendimiento
print(f"Mejores hiperparámetros: {random_search.best_params_}")
print(f"Mejor puntuación de validación (neg_mse): {random_search.best_score_}")

# Obtener el mejor modelo de RandomizedSearchCV
best_model = random_search.best_estimator_

# Realizar predicciones sobre el conjunto de prueba
y_pred = best_model.predict(X_test)

# Calcular el MSE
mse = mean_squared_error(y_test, y_pred)

# Calcular el RMSE
rmse = np.sqrt(mse)

# Mostrar el RMSE
print(f"RMSE del modelo optimizado con RandomizedSearch: {rmse}")





Fitting 5 folds for each of 4 candidates, totalling 20 fits
Mejores hiperparámetros: {'regression__fit_intercept': True, 'regression__copy_X': True}
Mejor puntuación de validación (neg_mse): 0.9995200166811612
RMSE del modelo optimizado con RandomizedSearch: 14.429764710724577


Aplicando GridSearch y RandomizarGS, no hubo mejoras en la busqueda de hiperparametros. Sin embargo, el modelo tiene una buena puntuación de validación.